In [1]:
### Pacotes necessarios:
# install.packages("e1071")
# install.packages("caret")
# install.packages("kernlab")
# install.packages("Metrics")
library("kernlab")
library("caret")
library(mlbench)
library("Metrics")

Carregando pacotes exigidos: ggplot2


Anexando pacote: 'ggplot2'


O seguinte objeto é mascarado por 'package:kernlab':

    alpha


Carregando pacotes exigidos: lattice


Anexando pacote: 'Metrics'


Os seguintes objetos são mascarados por 'package:caret':

    precision, recall




In [2]:
#carregamento da base de dados]
dados <- read.csv("databases/3 - Alunos - Dados.csv")
head(dados)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,⋯,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,GP,2,18,U,GT3,A,4,4,at_home,teacher,⋯,4,3,4,1,1,3,6,5,6,6
2,GP,2,17,U,GT3,T,1,1,at_home,other,⋯,5,3,3,1,1,3,4,5,5,6
3,GP,2,15,U,LE3,T,1,1,at_home,other,⋯,4,3,2,2,3,3,10,7,8,10
4,GP,2,15,U,GT3,T,4,2,health,services,⋯,3,2,2,1,1,5,2,15,14,15
5,GP,2,16,U,GT3,T,3,3,other,other,⋯,4,3,2,1,2,5,4,6,10,10
6,GP,1,16,U,LE3,T,4,3,services,other,⋯,5,4,2,1,2,5,10,15,15,15


In [3]:
#separação da base em teste e treino
set.seed(2034)
indices <- createDataPartition(dados$G3, p = 0.8, list = FALSE)

treino <- dados[indices, ]
teste <- dados[-indices, ]

In [4]:
#treinando com a base de treino
set.seed(2034)
svm <- train(G3 ~ ., data = treino, method = "svmRadial")
svm

Support Vector Machines with Radial Basis Function Kernel 

318 samples
 32 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 318, 318, 318, 318, 318, 318, ... 
Resampling results across tuning parameters:

  C     RMSE      Rsquared   MAE     
  0.25  2.773385  0.7182366  1.734741
  0.50  2.521874  0.7397979  1.587192
  1.00  2.369482  0.7514286  1.543962

Tuning parameter 'sigma' was held constant at a value of 0.01357067
RMSE was used to select the optimal model using the smallest value.
The final values used for the model were sigma = 0.01357067 and C = 1.

In [5]:
#aplicando o modelo e calculando as metricas
predict_svm <- predict(svm, treino)
rmse(treino$G3, predict_svm)

r2 <- function(observado, predito) {
  return(1- (sum((predito - observado)^2) /sum((observado - mean(observado))^2)))
}
r2(treino$G3, predict_svm)

[1] 1.609356

[1] 0.8732146

In [6]:
#experimentando com cross-validation
ctrl <- trainControl(method = "cv", number = 10)

set.seed(2034)
svm <- train(G3 ~ ., data = treino, method = "svmRadial", trControl = ctrl)
svm

Support Vector Machines with Radial Basis Function Kernel 

318 samples
 32 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 287, 287, 287, 286, 284, 286, ... 
Resampling results across tuning parameters:

  C     RMSE      Rsquared   MAE     
  0.25  2.658240  0.7729480  1.704507
  0.50  2.374741  0.7903106  1.505087
  1.00  2.181629  0.8039500  1.417870

Tuning parameter 'sigma' was held constant at a value of 0.01357067
RMSE was used to select the optimal model using the smallest value.
The final values used for the model were sigma = 0.01357067 and C = 1.

In [8]:
#aplicando o modelo na base de teste
predict_svm <- predict(svm, teste)

rmse(teste$G3, predict_svm)
r2(teste$G3, predict_svm)

[1] 2.235185

[1] 0.783011

In [9]:
# experimentando com outros valores de C e sigma
tune_grid <- expand.grid(C = c(1, 2, 10, 50, 100), sigma = c(0.01, 0.015, 0.2))

set.seed(2034)
svm <- train(G3 ~ ., data = treino, method = "svmRadial", trControl = ctrl, tuneGrid = tune_grid)
svm

Support Vector Machines with Radial Basis Function Kernel 

318 samples
 32 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 287, 287, 287, 286, 284, 286, ... 
Resampling results across tuning parameters:

  C    sigma  RMSE      Rsquared   MAE     
    1  0.010  2.142509  0.8105318  1.361722
    1  0.015  2.203408  0.8003739  1.445668
    1  0.200  4.490995  0.1192464  3.341135
    2  0.010  2.089425  0.8094689  1.354256
    2  0.015  2.188006  0.7925269  1.479533
    2  0.200  4.470187  0.1214253  3.365010
   10  0.010  2.218589  0.7765772  1.574266
   10  0.015  2.249536  0.7688360  1.629601
   10  0.200  4.470027  0.1208664  3.372695
   50  0.010  2.257976  0.7662019  1.661094
   50  0.015  2.252927  0.7665329  1.652268
   50  0.200  4.470027  0.1208664  3.372695
  100  0.010  2.257976  0.7662019  1.661094
  100  0.015  2.252927  0.7665329  1.652268
  100  0.200  4.470027  0.1208664  3.372695

RMSE was used to select the optimal model usi

In [10]:
#aplicando o modelo na base de teste e verificando metricas
predict_svm <- predict(svm, teste)

rmse(teste$G3, predict_svm)
r2(teste$G3, predict_svm)

[1] 2.114261

[1] 0.8058542

In [11]:
# predicao de novos casos
dados_novos_casos <- read.csv("databases/3 - Alunos - Novos Casos.csv")
View(dados_novos_casos)
dados_novos_casos$G3 <- NULL
predict_svm <- predict(svm, dados_novos_casos)
resultado <- cbind(dados_novos_casos, predict_svm)
View(resultado)

school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,⋯,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
GP,2,16,R,GT3,A,4,4,at_home,teacher,⋯,4,3,1,1,1,3,6,2,3,?
GP,1,17,U,GT3,T,1,1,at_home,other,⋯,5,3,3,1,1,3,4,15,15,?
GP,1,18,U,LE3,T,1,1,at_home,other,⋯,4,3,2,2,3,3,10,10,10,?


school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,⋯,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,predict_svm
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
GP,2,16,R,GT3,A,4,4,at_home,teacher,⋯,4,3,1,1,1,3,6,2,3,3.437812
GP,1,17,U,GT3,T,1,1,at_home,other,⋯,5,3,3,1,1,3,4,15,15,15.238994
GP,1,18,U,LE3,T,1,1,at_home,other,⋯,4,3,2,2,3,3,10,10,10,10.979451
